<a href="https://colab.research.google.com/github/cjfghk5697/anomaly-detection-competition/blob/main/Add_valid_transforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기록

## 07-04
기본적인 transforms valid 추가. 바뀐점은 밑에 기록해놓는다.


## transfors 추가
```python
class Custom_dataset(Dataset):
    def __init__(self, 
                 img_paths, 
                 labels, 
                 mode='train',
                 transforms= Sequence[Callable]
            ) -> None:
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        if self.transforms is not None:
            img = self.transforms(img)        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x
```

```python
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])
```
## valid 추가
```python
batch_size = 32
epochs = 25

# Train
trainset  = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train', transforms=transforms_train)
lengths = [int(len(trainset)*0.8), int(len(trainset)*0.2)]


train_dataset, valid_dataset = torch.utils.data.random_split(trainset, lengths)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, shuffle=True, batch_size=batch_size)
```

```python
    model.eval()
    with torch.no_grad():
      for i, (images, targets) in enumerate(valid_loader):

          images = images.to(device)
          targets = targets.to(device)

          outputs = model(images)
          valid_loss = criterion(outputs, targets)
      
          outputs = outputs > 0.5
          valid_acc = (outputs == targets).cpu().float().mean()
          valid_acc_list.append(valid_acc)
          valid_loss_list.append(valid_loss)
          print(f'{epoch}: "val loss:"{valid_loss.item():.5f}, "val acc: "{valid_acc.item():.5f}')        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'VALID    loss:"{valid_loss.item():.5f},    acc : "{valid_acc.item():.5f}')
    torch.save(model.state_dict(), f'{train_loss}_{train_f1}.pth')
```

## 뜬 error
```python
ValueError: Sum of input lengths does not equal the length of the input dataset!
```
해결법은 valid set으로 나눌때 length 배열을 보면 조금 부족하다.
원래 총 길이는 4277인데 lengths의 합은 4276이다. 왜냐하면 아마도 파이썬 특성중 하나인 완벽하게 정수를 반환하지 않는다. 원래 완벽한 계산을 안하는 특성이 있던걸로 기억난다. 그래서 이런 결과가 나온거같다. 간단하게 lengths[1]에 1을 더해 해결했다.


# 이런 작업은 이제 ez하다


### 라이브러리 불러오기

https://dacon.io/competitions/official/235894/overview/description

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/input/"
!unzip -q "/content/drive/MyDrive/input/train.zip" 

/content/drive/MyDrive/input


In [3]:
!pip3 install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 5.7 MB/s 


In [4]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random
from typing import Tuple, Sequence, Callable

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time
from PIL import Image
import cv2


device = torch.device('cuda')

In [5]:
train_png = sorted(glob('./train/*.png'))

In [6]:
train_y = pd.read_csv('./train_df.csv')

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [7]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img

In [8]:
train_imgs = [img_load(m) for m in tqdm(train_png)]

100%|██████████| 4277/4277 [02:54<00:00, 24.44it/s]


In [9]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])


In [10]:
class Custom_dataset(Dataset):
    def __init__(self, 
                 img_paths, 
                 labels, 
                 mode='train',
                 transforms= Sequence[Callable]
            ) -> None:
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        if self.mode=='test':
            pass
        img = Image.fromarray(img) # NumPy array to PIL image
        if self.transforms is not None:
            img = self.transforms(img)        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [11]:
batch_size = 32
epochs = 25

# Train
trainset  = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train', transforms=transforms_train)
lengths = [int(len(trainset)*0.8), int(len(trainset)*0.2)]


In [12]:

lengths=[lengths[0],lengths[1]+1]

In [13]:
train_dataset, valid_dataset = torch.utils.data.random_split(trainset, lengths)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, shuffle=True, batch_size=batch_size)

### 모델 학습

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 


valid_loss_list=[]
train_loss_list=[]

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]

    valid_loss = 0
    valid_pred=[]
    valid_y=[]

    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    model.eval()
    with torch.no_grad():
      for i in (valid_loader):
          images = torch.tensor(batch[0], dtype=torch.float32, device=device)
          targets = torch.tensor(batch[1], dtype=torch.long, device=device)



          outputs = model(images)
          loss = criterion(outputs, targets)

          valid_loss += loss.item()/len(valid_loader)
          valid_pred += outputs.argmax(1).detach().cpu().numpy().tolist()
          valid_y += targets.detach().cpu().numpy().tolist()
    
    train_f1 = score_function(train_y, train_pred)
    valid_f1 = score_function(valid_y, valid_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'valid    loss : {valid_loss:.5f}    f1 : {valid_f1:.5f}')

    torch.save(model.state_dict(), f'{train_loss}_{train_f1}.pth')

epoch : 1/25    time : 82s/1961s
TRAIN    loss : 1.10204    f1 : 0.19370
valid    loss : 0.38223    f1 : 0.77778
epoch : 2/25    time : 80s/1839s
TRAIN    loss : 0.62112    f1 : 0.30558
valid    loss : 0.60512    f1 : 0.59216
